In [1]:
%pwd

'e:\\Guvi_projects\\Smart_premium\\research'

In [2]:
import os
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir : Path
    

In [4]:
from src.Smart_premium_prediction.constants import *
from src.Smart_premium_prediction.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        ) 
        
        return data_ingestion_config

In [11]:
import os
import gdown
from src.Smart_premium_prediction import logger
import zipfile

In [12]:
class DataIngestion:
    def __init__(self,config:DataIngestionconfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading from: {self.config.source_URL}")
            gdown.download(url=self.config.source_URL, output=self.config.local_data_file, quiet=False)
            logger.info(f"Downloaded file to: {self.config.local_data_file}")
        else:
            logger.info("File already exists.")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-13 16:09:25,223: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-13 16:09:25,227: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-13 16:09:25,232: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-13 16:09:25,236: INFO: common: created directory at: artifacts]
[2025-05-13 16:09:25,253: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-13 16:09:25,256: INFO: 1075240264: Downloading from: https://drive.google.com/uc?id=1vTZuGIiguNiOZg9jBQxquA51cAlJBN40]


Downloading...
From (original): https://drive.google.com/uc?id=1vTZuGIiguNiOZg9jBQxquA51cAlJBN40
From (redirected): https://drive.google.com/uc?id=1vTZuGIiguNiOZg9jBQxquA51cAlJBN40&confirm=t&uuid=88fccfa3-6736-44b3-9073-802fbbf1dc99
To: e:\Guvi_projects\Smart_premium\artifacts\data_ingestion\data.zip
100%|██████████| 79.1M/79.1M [00:14<00:00, 5.65MB/s]


[2025-05-13 16:09:45,266: INFO: 1075240264: Downloaded file to: artifacts/data_ingestion/data.zip]
